In [1]:
#dependencies
import os
import h5py
import pylab as pl
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy

import utils.conversions as cnv
import utils.plotutils as plu
import utils.movie_plotutils as mplu
import utils.imreadin as imr
import utils.readMov as rdm
import utils.movie_aec as aec
import utils.train_model as tm

import imp
imp.reload(rdm)
imp.reload(tm)
imp.reload(cnv)
imp.reload(mplu)
from random import shuffle

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'dpi' : 500} #plotting pretty

/home/vasha/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
imp.reload(rdm)
movie_name = 'snow_monkeys' #'snow_monkeys' #
pixelpatchsize = 6
framepatchsize = 6

colorinvert = False

#m = imr.load_images('vh_corr', patch_edge_size=8)
#del m
with tf.device('/cpu:0'):
    try:
        m
        print('Movie Already Loaded.')
    except NameError:
        m = rdm.get_movie(movie_name,
                          pixel_patch_size = pixelpatchsize,
                          frame_patch_size = framepatchsize
                          )
        #m = np.transpose(m, (0, 3, 1, 2)) #change axis to [batchsize, time_patchsize, x_patchsize, y_patchsize]

        print("Movie Loaded. Shape is " + str(np.shape(m)))
        
        #only make anew dict and change these in params dict if we have reloaded images.
        params = {} #make a new dictionary
        #fill up our new dictionary
        params["movie_name"] = movie_name
        params['nmovies'] = np.shape(m)[0]
        params["pixelpatchsize"] = pixelpatchsize
        params["framepatchsize"] = framepatchsize
print("Done")

(120, 676, 1280)
normalizing movie...
making patches...
(477120, 6, 6, 6)
Movie Loaded. Shape is (477120, 6, 6, 6)
Done


In [14]:
#fixed parameters
params['frames_per_channel'] = params["framepatchsize"]  #OR convolution window (number of frames that project on one channel)
params['clipvec_len'] = params['frames_per_channel']*params['pixelpatchsize']*params['pixelpatchsize']

params["batchsize"] = 10000
params["learning_rate"] = 0.001

params["iterations"] = np.int(np.floor(params["nmovies"]/params["batchsize"]))
params["epochs"] = 4000


params['nonlinearity'] = 'relu'

params['nneurons'] = 400
#pixelpatchsize**2 * framepatchsize

#params for sweeping
noise_xs_rs_pairs = [[0.,0.]] #, [1e-1, 5], [1e-1, 2], [1e-1, 3.5], [1e-3,5e-1]]


#check to make sure our training numbers are consistent with our dataset
params["moviesperepoch"] = params["iterations"] *params["batchsize"]
if(params["moviesperepoch"] > params['nmovies']):
    print('ERROR! Trying to train on', params["moviesperepoch"],'movies per epoch, but we only have',params['nmovies'],'movie clips!')
else:
    print('Training',params["moviesperepoch"],'out of',params["nmovies"],'total movies.')
    
print('Compression Level: {}'.format((params["pixelpatchsize"]**2*params['frames_per_channel'])/params["nneurons"]))

Training 470000 out of 477120 total movies.
Compression Level: 0.54


In [ ]:
#reload
imp.reload(aec)
imp.reload(tm)
imp.reload(mplu)
imp.reload(plu)

for xs,rs in noise_xs_rs_pairs:
    params['noise_x'] = xs
    params['noise_r'] = rs

    #make our model
    aecmodel = aec.movie_aec_model(params)

    #train it
    [cost_evolution,
     activation_evolution,
     inweights_evolution,
     outweights_evolution,
     clips,
     recons,
     final_inweights,
     final_outweights,
     final_activations] = tm.train_movie_model(aecmodel, m)

    #mplu.save_plots(aecmodel,
    #                cost_evolution,
    #                activation_evolution,
    #                inweights_evolution,
    #                outweights_evolution,
    #                clips,
    #                recons,
    #                final_inweights,
    #                final_outweights,
    #                final_activations)

    print('All Done!')

Compressing by 0.09 for a total of 400 neurons
400 hidden neurons, noise_in at 0.0, noise_out at 0.0
Training 47 iterations in 4000 epochs... 


Epoch 0/4000 Epoch 1/4000 Epoch 2/4000 Epoch 3/4000 Epoch 4/4000 Epoch 5/4000 Epoch 6/4000 Epoch 7/4000 Epoch 8/4000 Epoch 9/4000 Epoch 10/4000 Epoch 11/4000 Epoch 12/4000 Epoch 13/4000 Epoch 14/4000 Epoch 15/4000 Epoch 16/4000 Epoch 17/4000 Epoch 18/4000 Epoch 19/4000 Epoch 20/4000 Epoch 21/4000 Epoch 22/4000 Epoch 23/4000 Epoch 24/4000 Epoch 25/4000 Epoch 26/4000 Epoch 27/4000 Epoch 28/4000 Epoch 29/4000 Epoch 30/4000 Epoch 31/4000 Epoch 32/4000 Epoch 33/4000 Epoch 34/4000 Epoch 35/4000 Epoch 36/4000 Epoch 37/4000 Epoch 38/4000 Epoch 39/4000 Epoch 40/4000 Epoch 41/4000 Epoch 42/4000 Epoch 43/4000 Epoch 44/4000 Epoch 45/4000 Epoch 46/4000 Epoch 47/4000 Epoch 48/4000 Epoch 49/4000 Epoch 50/4000 Epoch 51/4000 Epoch 52/4000 Epoch 53/4000 Epoch 54/4000 Epoch 55/4000 Epoch 56/4000 Epoch 57/4000 Epoch 58/4000 Epoch 59/4000 Epoch 60/4000 Epoch 61/4000 Epoch 62/4000 Epoch 63/4000 Epoch 64/4000 Epoch 65/4000 Epoch 66/4000 Epoch 67/4000 Epoch 68/4000 Epoch 69/4000 Epoch 70/4000 Epoch 71/4000 Ep

Epoch 554/4000 Epoch 555/4000 Epoch 556/4000 Epoch 557/4000 Epoch 558/4000 Epoch 559/4000 Epoch 560/4000 Epoch 561/4000 Epoch 562/4000 Epoch 563/4000 Epoch 564/4000 Epoch 565/4000 Epoch 566/4000 Epoch 567/4000 Epoch 568/4000 Epoch 569/4000 Epoch 570/4000 Epoch 571/4000 Epoch 572/4000 Epoch 573/4000 Epoch 574/4000 Epoch 575/4000 Epoch 576/4000 Epoch 577/4000 Epoch 578/4000 Epoch 579/4000 Epoch 580/4000 Epoch 581/4000 Epoch 582/4000 Epoch 583/4000 Epoch 584/4000 Epoch 585/4000 Epoch 586/4000 Epoch 587/4000 Epoch 588/4000 Epoch 589/4000 Epoch 590/4000 Epoch 591/4000 Epoch 592/4000 Epoch 593/4000 Epoch 594/4000 Epoch 595/4000 Epoch 596/4000 Epoch 597/4000 Epoch 598/4000 Epoch 599/4000 Epoch 600/4000 Epoch 601/4000 Epoch 602/4000 Epoch 603/4000 Epoch 604/4000 Epoch 605/4000 Epoch 606/4000 Epoch 607/4000 Epoch 608/4000 Epoch 609/4000 Epoch 610/4000 Epoch 611/4000 Epoch 612/4000 Epoch 613/4000 Epoch 614/4000 Epoch 615/4000 Epoch 616/4000 Epoch 617/4000 Epoch 618/4000 Epoch 619/4000 Epoch 620/

In [ ]:
print(final_inweights.shape)
print(final_outweights.shape)

In [ ]:
plt.hist(final_outweights.flatten())

In [ ]:
plt.plot(np.log(np.mean(cost_evolution,axis=2).flatten()))

In [ ]:
print(final_inweights.shape)
print(final_outweights.shape)
plu.plot_tiled_rfs(final_inweights[-1].T)
plt.show()
plu.plot_tiled_rfs(final_outweights[...,-1])

In [ ]:
#test = final_outweights.T
#print(np.shape(test))
print(final_outweights.shape)
mplu.plot_temporal_weights(final_outweights[...,0].T);

In [ ]:
imp.reload(mplu)
imp.reload(plu)
#plu.plot_tiled_rfs(np.rollaxis(inweights_evolution[-1][0,:,:,:],2,0))
#mplu.plot_temporal_weights(np.rollaxis(inweights_evolution[-1][0,:,:,:],2,0))

#print(inweights_evolution.shape)
#test = inweights_evolution[-1].T
test = outweights_evolution[-1]
print(test.shape)
#test = np.rollaxis(test[0,:,:,:],2,0)
plu.plot_tiled_rfs(test)
#print(test)
#mplu.plot_temporal_weights(final_outweights);
#plu.plot_tiled_rfs()

In [13]:
imp.reload(mplu)
imp.reload(plu)
print(final_inweights.shape)
#mplu.plot_temporal_weights(final_inweights);
#plu.plot_tiled_rfs(np.rollaxis(final_inweights[0,:,:,:],2,0))

#plt.figure(figsize=(10,3))
#test = np.transpose(final_inweights,axes=(0,2,1,3))
#print(test.shape)
#mplu.plot_temporal_weights(final_outweights.T);
mplu.plot_temporal_weights(final_inweights);
#plu.plot_tiled_rfs(np.rollaxis(test[0,:,:,:],2,0))


(6, 6, 6, 400)


TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
imp.reload(mplu)
animation = mplu.weights_movieclip(final_outweights.T);
#animation.ipython_display(fps=20, loop=True, autoplay=True)
#from IPython.display import HTML
#HTML('<img src="./weight_movie.gif>')

In [ ]:
print(final_inweights[-1].shape)
p = plu.pad_data(final_inweights[-1].T)
plt.imshow(p,cmap='Greys_r')

In [ ]:
mplu.plot_temporal_weights(np.transpose(final_inweights,axes=(0,2,1,3)));

In [ ]:
##TWO BASIS VECTORS##
mplu.plot_mov_recon(final_outweights.T[:,:,:,8],final_outweights.T[:,:,:,20],nframes=params['frames_per_channel']);


In [ ]:
##TWO BASIS VECTORS##
mplu.plot_mov_recon(test[2,:,:,:],final_inweights[2,:,:,:],nframes=params['frames_per_channel']);


In [ ]:
#diff = final_inweights - inweights_evolution[1]
#print(diff.shape)
print(final_inweights.shape)
for i in range(final_inweights.shape[3]):
    sbplt = np.sqrt(final_inweights.shape[3])
    plt.subplot(sbplt,sbplt,i+1)
    plt.imshow(np.linalg.inv(final_outweights.T[0,:,:,i]),cmap='Greys_r')
    #plt.imshow(final_inweights[2,:,:,i],cmap='Greys_r')
    #plt.imshow(test,cmap='Greys_r')
    plt.xticks([])
    plt.yticks([])


In [ ]:
print(final_inweights.shape)
print(final_outweights.shape)
print(np.array(recons).shape)
print(np.array(clips).shape)

In [ ]:
randomclipindex = np.random.randint(np.shape(clips[0])[0])

#print the cost of this instance
print(np.mean(np.log1p(clips[-1][randomclipindex,:,:,:] - recons[-1][randomclipindex,:,:,:])))

mplu.plot_mov_recon(clips[-1][randomclipindex,:,:,:],recons[-1][randomclipindex,:,:,:],nframes=5);

In [ ]:
##Check norm of weights
plt.plot(np.linalg.norm(final_inweights.reshape(8*64,1000), ord=2, axis=0))

In [ ]:
print(inweights_evolution[-1].T.shape)
#plu.plot_tiled_rfs(inweights_evolution[-1].T[:,:,:,0])
padded_wv = plu.pad_data(inweights_evolution[-1].T[:,:,:,0])
print(padded_wv.shape)
plt.imshow(padded_wv)

In [ ]:
import imageio
images = []

for i in range(np.shape(final_inweights)[3]):
    basisfun = plt.imshow(final_inweights[0,:,:,i],cmap='Greys_r');
    images.append(basisfun)
imageio.mimsave('test.gif', images)

In [ ]:
p = plt.errorbar(np.arange(len(cost_evolution)),
                 np.log(np.mean(np.array(cost_evolution),axis=1)),
                 yerr=np.log(np.std(np.array(cost_evolution),axis=1)));
plt.title("Log-Cost Evolution Over Iterations (Errorbar = 1SD)")

In [ ]:
np.shape(np.array(activation_evolution))

In [ ]:
p = plt.errorbar(np.arange(len(activation_evolution)),
                 np.array(activation_evolution),
                 yerr=np.std(np.array(activation_evolution),axis=0));
plt.title('Evolution of Activation over Learning')

In [ ]:
plt.imshow(recons[0][randomclipindex],cmap="Greys_r")
plt.colorbar()

In [ ]:
plt.imshow(clips[0][randomclipindex],cmap="Greys_r")
plt.colorbar()

In [ ]:

np.shape(final_inweights)

In [ ]:
np.shape(inweights_evolution)

In [ ]:
import sklearn as skl

npatches = 1000

test = m[0:npatches,:5,:,:].reshape((npatches,-1))
print(test.shape)
cov_mat = np.cov(test)
eig_val, eig_vec = np.linalg.eigh(cov_mat)
print(eig_val)
print(np.shape(eig_val))

invcovmat = eig_vec
#recon_val, recon_vec = np.linalg.eig(invcovmat)

#recon_vec

#pca = skl.decomposition.PCA()
#pca.fit(test)
#np.pca(test).shape
#pca